In [ ]:
# Esercizi da fare:
# Remove 5 landmarks and see the result
# Play with the noise (odometry and sensor)
# Play with the number of particles [10, 250, 500, 1000]

#Credits to Thrun, S. Particle Filters in Robotics. Proceedings of the 17th Annual Conference on Uncertainty in AI (UAI), 2002.
import numpy as np
import matplotlib.pyplot as plt
from math import *
import random

landmarks = [[20.0, 20.0], [20.0, 80.0], [20.0, 50.0],
             [50.0, 20.0], [50.0, 80.0], [80.0, 80.0],
             [80.0, 20.0], [80.0, 50.0]]
world_size = 100.0 #100x/100y

class robot:

    #initialize the position of the robot
    def __init__(self):
        self.x = random.random() * world_size
        self.y = random.random() * world_size
        self.orientation = random.random() * 2.0 * pi
        self.forward_noise = 0.0
        self.turn_noise    = 0.0
        self.sense_noise   = 0.0
        self.weight        = 1.0

    #set initial position of the robot
    def set(self, new_x, new_y, new_orientation):
        if new_x < 0 or new_x >= world_size:
            raise ValueError('X coordinate out of bound')
        if new_y < 0 or new_y >= world_size:
            raise ValueError('Y coordinate out of bound')  # Correct syntax
        if new_orientation < 0 or new_orientation >= 2 * pi:
            raise ValueError('Orientation must be in [0..2pi]')  # Correct syntax
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)

    #noise new position of the robot
    def set_noise(self, new_f_noise, new_t_noise, new_s_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.forward_noise = float(new_f_noise);
        self.turn_noise    = float(new_t_noise);
        self.sense_noise   = float(new_s_noise);

    #distance of the robot wrt the landmarks
    def sense(self):
        Z = []
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            dist += random.gauss(0.0, self.sense_noise)
            Z.append(dist)
        return Z

    #move the robot according to the function parameters
    def move(self, turn, forward):
        if forward < 0:
            raise (ValueError, 'Robot cant move backwards')

        # turn, and add randomness to the turning command
        orientation = self.orientation + float(turn) + random.gauss(0.0, self.turn_noise)
        orientation %= 2 * pi

        # move, and add randomness to the motion command
        dist = float(forward) + random.gauss(0.0, self.forward_noise)
        x = self.x + (cos(orientation) * dist)
        y = self.y + (sin(orientation) * dist)
        x %= world_size    # cyclic truncate
        y %= world_size

        # set particle
        res = robot()
        res.set(x, y, orientation)
        res.set_noise(self.forward_noise, self.turn_noise, self.sense_noise)
        return res

    # calculates the probability of x for 1-dim Gaussian with mean mu and var. sigma
    def Gaussian(self, mu, sigma, x):
        return exp(- ((mu - x) ** 2) / (sigma ** 2) / 2.0) / sqrt(2.0 * pi * (sigma ** 2))


    def measurement_prob(self, measurement):
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            self.weight *= self.Gaussian(dist, self.sense_noise, measurement[i])
        return self.weight

    def __repr__(self):
        return '[x=%.6s y=%.6s orient=%.6s]' % (str(self.x), str(self.y), str(self.orientation))




: 

In [ ]:
def eval(r, p):
    sum = 0.0;
    for i in range(len(p)): # calculate mean error
        dx = (p[i].x - r.x + (world_size/2.0)) % world_size - (world_size/2.0)
        dy = (p[i].y - r.y + (world_size/2.0)) % world_size - (world_size/2.0)
        err = sqrt(dx * dx + dy * dy)
        sum += err
    return sum / float(len(p))


#### First let's move the robot!
def moveRobot():
    myrobot = robot()
    print ("create myrobot: ", myrobot)
    myrobot.set(30.0, 50.0, pi/2.0)
    positionsX=[]
    positionsY=[]
    orientation=[]

    plt.axis([0, 100, 0, 100])
    positionsX.append(myrobot.x)
    positionsY.append(myrobot.y)
    orientation.append(myrobot.orientation)

    print ("set myrobot: ", myrobot)
    myrobot = myrobot.move(-pi/2.0, 15.0)
    positionsX.append(myrobot.x)
    positionsY.append(myrobot.y)
    orientation.append(myrobot.orientation)

    print ("move: ", myrobot)
    print ("sense: ", myrobot.sense())
    myrobot = myrobot.move(-pi/2.0, 10.0)
    positionsX.append(myrobot.x)
    positionsY.append(myrobot.y)
    orientation.append(myrobot.orientation)

    print ("move: ", myrobot)
    print ("sense: ", myrobot.sense())
    for i in range(len(positionsX)):
      plt.plot(positionsX[i], positionsY[i], marker=(3, 0, orientation[i]* 180 / pi), markersize=20, linestyle='None')

    for l in landmarks:
      plt.scatter(l[0], l[1],color='red', s=100)

    plt.show()


moveRobot()

In [ ]:
#### The robot can turn and we can model the noise!
def moveRobotTurn():
    positionsX=[]
    positionsY=[]
    orientation=[]
    myrobot = robot()
    myrobot.set_noise(5.0, 0.1, 5.0)
    myrobot.set(30.0, 50.0, pi/2)
    positionsX.append(myrobot.x)
    positionsY.append(myrobot.y)
    orientation.append(myrobot.orientation)
    plt.axis([0, 100, 0, 100])
    print ("move: ", myrobot)

    myrobot = myrobot.move(-pi/2, 15.0)
    print ("move: ", myrobot)
    #print (myrobot.sense())
    positionsX.append(myrobot.x)
    positionsY.append(myrobot.y)
    orientation.append(myrobot.orientation)

    myrobot = myrobot.move(-pi/2, 10.0)
    #print (myrobot.sense())
    print ("move: ", myrobot)
    positionsX.append(myrobot.x)
    positionsY.append(myrobot.y)
    orientation.append(myrobot.orientation)

    for i in range(len(positionsX)):
      plt.plot(positionsX[i], positionsY[i], marker=(3, 0, orientation[i]* 180 / pi), markersize=20, linestyle='None')
    for l in landmarks:
        plt.scatter(l[0], l[1],color='red', s=100)
    plt.show()

moveRobotTurn()


In [11]:
def visualization(robot, step, p, pr, weights):
    '''
    :param robot:   the current robot object
    :param step:    the current step
    :param p:       list with particles
    :param pr:      list of resampled particles
    :param weights: particle weights
    '''
    plt.figure("Robot in the world ", figsize=(15., 15.))
    # draw coordinate grid for plotting
    grid = [0, world_size, 0, world_size]
    plt.axis(grid)
    #plt.grid(b=True, which='major', color='0.75', linestyle='--')
    plt.xticks([i for i in range(0, int(world_size), 5)])
    plt.yticks([i for i in range(0, int(world_size), 5)])

    # Trova l'indice della particella con il peso massimo
    max_weight_index = weights.index(max(weights))

    # draw particles
    for ind in range(len(p)):

        if ind == max_weight_index:
            # Disegna la particella con il peso massimo in rosso
            circle = plt.Circle((p[ind].x, p[ind].y), 2., facecolor='#050505', edgecolor='#050505', alpha=0.7) # black
            arrow = plt.Arrow(p[ind].x, p[ind].y, 2*cos(p[ind].orientation), 2*sin(p[ind].orientation), alpha=1., facecolor='#cc0000', edgecolor='#cc0000')
        else:
            # Disegna tutte le altre particelle
            circle = plt.Circle((p[ind].x, p[ind].y), 1., facecolor='#ffb266', edgecolor='#994c00', alpha=0.5) # yellow
            arrow = plt.Arrow(p[ind].x, p[ind].y, 2*cos(p[ind].orientation), 2*sin(p[ind].orientation), alpha=1., facecolor='#994c00', edgecolor='#994c00')

        plt.gca().add_patch(circle)
        plt.gca().add_patch(arrow)

    # draw resampled particles
    for ind in range(len(pr)):

        # particle
        circle = plt.Circle((pr[ind].x, pr[ind].y), 1., facecolor='#66ff66', edgecolor='#009900', alpha=0.5) # green
        plt.gca().add_patch(circle)

        # particle's orientation
        arrow = plt.Arrow(pr[ind].x, pr[ind].y, 2*cos(pr[ind].orientation), 2*sin(pr[ind].orientation), alpha=1., facecolor='#006600', edgecolor='#006600')
        plt.gca().add_patch(arrow)

    # fixed landmarks of known locations
    for lm in landmarks:
        circle = plt.Circle((lm[0], lm[1]), 1., facecolor='#cc0000', edgecolor='#330000')
        plt.gca().add_patch(circle)

    # robot's location
    circle = plt.Circle((robot.x, robot.y), 1., facecolor='#6666ff', edgecolor='#0000cc') # blue
    plt.gca().add_patch(circle)

    # robot's orientation
    arrow = plt.Arrow(robot.x, robot.y, 2*cos(robot.orientation), 2*sin(robot.orientation), alpha=0.5, facecolor='#000000', edgecolor='#000000')
    plt.gca().add_patch(arrow)

    #plt.savefig("figure_" + str(step) + ".png")
    plt.show()
    #plt.close()

In [ ]:

# We define multiples particles as a robot representation
def createParticlesTest():
    N = 100
    p = []
    plt.axis([0, 100, 0, 100])

    for i in range(N):
        p.append(robot())
    print (len(p))
    print (p)

    for i in range(N):
      plt.plot(p[i].x, p[i].y, marker=(3, 0, p[i].orientation* 180 / pi), markersize=10, linestyle='None')

    for l in landmarks:
        plt.scatter(l[0], l[1],color='red', s=100)

    plt.show()

createParticlesTest()

In [ ]:
# Particle filter without resampling
def particleFilter():
    myrobot = robot() # blue
    myrobot = myrobot.move(0.1, 5.0)
    myrobot.set(50.0, 50.0, pi/2.0)

    Z = myrobot.sense()

    N = 100
    p = []
    moveTimes=20

    #here we create the particles with noise (Initialization phase)
    for i in range(N):
        x = robot()
        x.set_noise(0.05, 0.05, 5.0)
        p.append(x)

    for i in range(moveTimes):
        myrobot = myrobot.move(0.1, 5.0)
        Z = myrobot.sense()

        #move all the particles
        p2 = []
        for i in range(N):
            p2.append(p[i].move(0.1, 5.0))
        p = p2

        #compute for each particle the probability of being in that position
        w = []
        for j in p:
            w.append(j.measurement_prob(Z))
        print (w)

        max_value = max(w) #orange
        max_value_index = w.index(max_value)
        visualization(myrobot,i, p2, p, w)


    ##print ("max_value "+str(max_value_index))

particleFilter()

In [ ]:
def fullParticleFilter(N = 50):
    myrobot = robot()
    moveTimes = 10
    p = []
    for i in range(N):
        x = robot()
        x.set_noise(0.05, 0.05, 5.0)
        p.append(x)

    for t in range(moveTimes):

        myrobot = myrobot.move(0.1, 5.0)
        Z = myrobot.sense()

        p2 = []
        for i in range(N):
              p2.append(p[i].move(0.1, 5.0))
        p = p2

        #print (eval(myrobot,p))

        w = []
        for i in range(N):
            w.append(p[i].measurement_prob(Z))

        p3 = []
        #Resampling wheel
        index = int(random.random() * N)
        beta = 0.0
        mw = max(w)
        for i in range(N):
            beta += random.random() * 2.0 * mw
            while beta > w[index]:
                beta -= w[index]
                index = (index + 1) % N
            p3.append(p[index])
        p=p3
        '''
        :param robot:   the current robot object
        :param step:    the current step
        :param p:       list with particles
        :param pr:      list of resampled particles
        :param weights: particle weights
        '''
        visualization(myrobot,t, p2, p, w)
        print(eval(myrobot,p))

# Resampled particles in green
# Particles in yellow
# Ground truth (robot's position) in blue
fullParticleFilter()

In [ ]:
landmarks2 = [[20.0, 20.0], [20.0, 80.0], [20.0, 50.0],
             [50.0, 20.0], [50.0, 80.0], [80.0, 80.0],
             [80.0, 20.0], [80.0, 50.0], [50.0, 50.0],
             [35.0, 65.0], [70.0,65.0], [35.0,35.0], [70.0,35.0]]

landmarks = [[20.0, 20.0], [20.0, 80.0], [20.0, 50.0],
             [50.0, 20.0], [50.0, 80.0], [80.0, 80.0],
             [80.0, 20.0], [80.0, 50.0]]
def fullParticleFilterImprovedResampling(N = 50):
    myrobot = robot()
    moveTimes = 10
    p = []
    for i in range(N):
        x = robot()
        x.set_noise(0.05, 0.5, 5.0)
        p.append(x)

    for t in range(moveTimes):

        myrobot = myrobot.move(0.1, 5.0)
        Z = myrobot.sense()

        p2 = []
        for i in range(N):
              p2.append(p[i].move(0.1, 5.0))
        p = p2

        #print (eval(myrobot,p))

        w = []
        for i in range(N):
            w.append(p[i].measurement_prob(Z))

        p3 = []
        #Resampling wheel
        index = int(random.random() * N)
        beta = 0.0
        mw = max(w)
        for i in range(N):
            beta += random.random() * 2.0 * mw
            while beta > w[index]:
                beta -= w[index]
                index = (index + 1) % N
            p3.append(p[index])

        # Aggiungi particelle rigenerate casualmente intorno alle migliori
        num_random_particles = int(0.1 * N)  # rigenera il 10% delle particelle
        best_particles = sorted(p3, key=lambda particle: particle.measurement_prob(Z), reverse=True)[:num_random_particles]

        for i in range(num_random_particles):
            # Seleziona una delle migliori particelle
            best_particle = random.choice(best_particles)

            # Crea una nuova particella intorno alla migliore aggiungendo rumore casuale
            random_particle = robot()
            random_particle.set_noise(0.05, 0.5, 1)
            random_particle.set(
                (best_particle.x + random.gauss(0, 0.5) )% world_size,  # piccolo spostamento casuale
                (best_particle.y + random.gauss(0, 0.5))% world_size,
                (best_particle.orientation + random.gauss(0, 100)) % (2 * pi)  # normalizza l'orientamento tra 0 e 2*pi
            )
            p3[i] = random_particle  # Sostituisci alcune particelle con quelle rigenerate

        p = p3
        '''
        :param robot:   the current robot object
        :param step:    the current step
        :param p:       list with particles
        :param pr:      list of resampled particles
        :param weights: particle weights
        '''
        visualization(myrobot,t, p2, p, w)
        print(eval(myrobot,p))

fullParticleFilterImprovedResampling(100)


Exercise 1: Analyze the effect of noise

    Objective: Experiment with different noise levels in the set_noise() function for both the motion and measurement models.
    Task: Test different noise settings and observe how it impacts the accuracy of the particle filter.
    Learning Goal: Learn how noise levels influence state estimation and the trade-off between exploration and convergence in the filter.

Exercise 2: Evaluate resampling strategy

    Objective: Modify the resampling wheel to use other resampling techniques, such as systematic resampling.
    Task: Replace the resampling wheel with systematic resampling and compare the results with the original wheel approach.
    Learning Goal: Explore the differences between various resampling techniques and their impact on particle filter performance.

Exercise 3: Optimize the number of random particles

    Objective: Experiment with the proportion of random particles added around the best particles.
    Task: Adjust the num_random_particles percentage (e.g., try 5%, 20%, etc.) and observe how it impacts the filter’s performance over time.
    Learning Goal: Understand the effect of regenerating particles around the best particles and how to balance exploitation vs. exploration.

Exercise 4: Vary the number of particles

    Objective: Modify the number of particles (N) and measure the filter’s performance with different population sizes.
    Task: Compare the performance of the filter when using small (N=10), medium (N=50), and large (N=200) particle sets.
    Learning Goal: Trade-offs between computational complexity and accuracy in particle filters.

Exercise 5: Implement particle depletion prevention

    Objective: Add logic to the algorithm that detects particle depletion and introduces mechanisms to regenerate particles when diversity is too low.
    Task: Introduce a condition to check for particle diversity (e.g., too many identical particles) and implement an adaptive mechanism to regenerate particles.
    Learning Goal: Understand and mitigate particle depletion, ensuring the particle filter maintains its performance over time.